In [64]:
import requests
import json
import os
import zipfile
import pandas as pd

In [18]:
def findDatesAvailable(dpID, site):
    server = 'https://data.neonscience.org/api/v0/'
    url = f'{server}sites/{site}'
    response = requests.get(url)
    data = response.json()['data']
    dataProducts = data['dataProducts']
    dates = None
    for dp in dataProducts:
        if dp['dataProductCode'] == dpID:
            dates = dp['availableMonths']
    return(dates)

In [19]:
dates = findDatesAvailable('DP3.30015.001', 'BART')

In [21]:
date = dates[-1]

In [87]:
def downLoadByProduct(dpID, site, date, savedir='data', metadict=False):
    '''
    Downloads files, places them within savedir in sub directories:
    savedir/site/dpID/date
    
    Optionaly returns metadata csvs as dataframes inside of a dict. The key for each df
    is the meaningful part of the filename.
    
    --------
    Parameters
    --------
    dpID     - String, Product ID, e.g. 'DP1.10098.001'
    site     - String, NEON site abreviation, e.g. 'BART' 
    date     - String, Month of data to download e.g. '2019-08'
    savedir  - String, optional, directory where files will be saved. Defaults to ./data
    metadict - Boolean, optional, If True returns dictionary with metadata. Defaults to False.
    '''
    savedir.rstrip('/')
    # make sure target directory exists 
    os.makedirs(savedir, exist_ok=True)
    # make sure data exists for date
    try:
        assert date in findDatesAvailable(dpID, site)
    except:    
        print(f'{dpID} is not available for {site} in the month{date}.\n Try using findDatesAvailable(dpID, site)')
    # See what files theree are
    try:
        server = 'https://data.neonscience.org/api/v0/'
        url = f'{server}data/{dpID}/{site}/{date}'
        response = requests.get(url)
    except Exception as e:
        print(e)
    stuff = response.json()
    # download the files
    files = []
    for f in stuff['data']['files']:
        fname = f['name']
        response = requests.get(f['url'])
        with open(f'{savedir}/{fname}', 'wb') as f:
            f.write(response.content)
        files.append(fname)
    data = [f for f in files if (date in f and 'zip' in f)]
    for f in data:
        os.makedirs(f'{savedir}/{site}/{dpID}/{date}', exist_ok=True)
        z= zipfile.ZipFile(f'{savedir}/{f}')
        z.extractall(savedir)
        z.close
    frames = dict()
    if metadict == True:
        for f in files:
            if f.endswith('.csv'):
                name = f.partition(dpID+'.')[-1].split('.')[0]
                if name:
                    # make a dataframe and put it in frames with the 
                    # important part of the filename as key
                    s = f'frames[name] = pd.read_csv(\'{savedir}/{f}\')'
                    exec(s)
        return(frames)
        

In [88]:
frames = downLoadByProduct('DP1.10098.001', 'BART', date, metadict=True)

In [94]:
ls data/BART/DP1.10098.001/

2019-08/


In [89]:
vst_mappingandtagging =frames['vst_mappingandtagging']

In [90]:
vst_mappingandtagging

,uid,namedLocation,date,eventID,domainID,siteID,plotID,subplotID,nestedSubplotID,pointID,...,scientificName,taxonRank,identificationReferences,morphospeciesID,morphospeciesIDRemarks,identificationQualifier,remarks,measuredBy,recordedBy,dataQF
0,08e9053f-0227-4f34-bf25-a77785f651c9,BART_012.basePlot.vst,2014-08-05,vst_BART_2014,D01,BART,BART_012,40.0,NaN,51.0,...,Fagus grandifolia Ehrh.,species,NaN,NaN,NaN,NaN,NaN,llukas@neoninc.org,ramundson@field-ops.org,NaN
1,e5037180-6d06-40c6-a3c7-2e1bcef32619,BART_012.basePlot.vst,2014-08-05,vst_BART_2014,D01,BART,BART_012,31.0,1.0,NaN,...,Fagus grandifolia Ehrh.,species,NaN,NaN,NaN,NaN,QC: NO ERRORS,llukas@neoninc.org,ramundson@field-ops.org,NaN
2,b5f9593c-c304-4cd9-a87d-ddbef9d34101,BART_012.basePlot.vst,2014-08-05,vst_BART_2014,D01,BART,BART_012,40.0,1.0,NaN,...,Fagus grandifolia Ehrh.,species,NaN,NaN,NaN,NaN,NaN,llukas@neoninc.org,ramundson@field-ops.org,NaN
3,99bcccc3-797b-4c95-8c55-8c679c796adf,BART_012.basePlot.vst,2014-08-05,vst_BART_2014,D01,BART,BART_012,31.0,1.0,NaN,...,Fagus grandifolia Ehrh.,species,NaN,NaN,NaN,NaN,NaN,llukas@neoninc.org,ramundson@field-ops.org,NaN
4,bc43df62-fce8-4d2e-8680-a23a7062c1b6,BART_012.basePlot.vst,2014-08-05,vst_BART_2014,D01,BART,BART_012,40.0,3.0,NaN,...,Fagus grandifolia Ehrh.,species,NaN,NaN,NaN,NaN,NaN,llukas@neoninc.org,ramundson@field-ops.org,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6198,ca61d820-334e-43a6-91eb-34ea1756fdaa,BART_019.basePlot.vst,2019-10-15,vst_BART_2019,D01,BART,BART_019,40.0,3.0,NaN,...,Abies sp.,genus,NaN,NaN,NaN,NaN,NaN,0000-0001-9529-4416,0000-0003-1933-3182,NaN
6199,14a836c8-fe6a-423e-a209-47d69518c738,BART_019.basePlot.vst,2019-10-15,vst_BART_2019,D01,BART,BART_019,40.0,NaN,49.0,...,Acer rubrum L.,species,NaN,NaN,NaN,NaN,NaN,0000-0003-1933-3182,0000-0001-9529-4416,NaN
6200,2609fe7e-c208-4ef1-b2fd-949e3118b65d,BART_019.basePlot.vst,2019-10-15,vst_BART_2019,D01,BART,BART_019,40.0,NaN,49.0,...,Betula papyrifera Marshall var. papyrifera,variety,NaN,NaN,NaN,NaN,NaN,0000-0003-1933-3182,0000-0001-9529-4416,NaN
6201,a9f85e3f-0414-4aba-8b6c-811861f93483,BART_019.basePlot.vst,2019-10-15,vst_BART_2019,D01,BART,BART_019,41.0,NaN,51.0,...,Acer rubrum L.,species,NaN,NaN,NaN,NaN,NaN,0000-0003-1933-3182,0000-0001-9529-4416,NaN


In [51]:
list(filter(None, [f.partition('DP1.10098.001'+'.')[-1].split('.')[0] for f in files]))

['vst_apparentindividual',
 'readme',
 '2019-08',
 'EML',
 'variables',
 'vst_mappingandtagging']